In [57]:
from transformers import pipeline
import torch
from textattack.augmentation import WordNetAugmenter, EmbeddingAugmenter, EasyDataAugmenter
import nlpaug.augmenter.word as naw
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [42]:
#Check if CuUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [17]:
# Creaate an classification analysis pipeline
pipe = pipeline("text-classification", model ="distilbert/distilbert-base-uncased-finetuned-sst-2-english", device=device)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [18]:
textToScramble = """
Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in
our lives. Hope you too, to enjoy it as my deepest wishes.
Thank your message to show our words to the doctor, as his next contract checking, to all of us.
I got this message to see the approved message. In fact, I have received the message from the
professor, to show me, this, a couple of days ago. I am very appreciated the full support of the
professor, for our Springer proceedings publication
During our final discuss, I told him about the new submission — the one we were waiting since
last autumn, but the updates was confusing as it not included the full feedback from reviewer or
maybe editor?
Anyway, I believe the team, although bit delay and less communication at recent days, they really
tried best for paper and cooperation. We should be grateful, I mean all of us, for the acceptance
and eﬀorts until the Springer link came finally last week, I think.
Also, kindly remind me please, if the doctor still plan for the acknowledgments section edit before
he sending again. Because I didn’t see that part final yet, or maybe I missed, I apologize if so.
Overall, let us make sure all are safe and celebrate the outcome with strong coﬀee and future
targets
"""

In [19]:
generator = pipeline("text-generation", model="gpt2")
textGenerationOutput = generator(textToScramble, max_new_tokens=512)[0]['generated_text']

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [48]:
augmenter = WordNetAugmenter()
augmentedText = augmenter.augment(textToScramble)

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


['\nToday is our dragon sauceboat festival, in our Chinese finish, to fete it with all prophylactic and great in\nour lives. Hope you too, to love it as my bass wishes.\nThank your message to point our words to the doctor, as his next contract fit, to all of us.\nI draw this message to see the approved message. In fact, I have received the message from the\nprofessor, to point me, this, a couple of days ago. I am very apprise the full patronize of the\nprofessor, for our Springer proceedings publication\nDuring our final discuss, I state him about the new submission — the one we were waiting since\nlast autumn, but the updates was confusing as it not admit the full feedback from reviewer or\nmaybe editor?\nAnyway, ane believe the team, although bit delay and less communication at recent days, they really\ntried secure for wallpaper and cooperation. We should be grateful, I think all of us, for the acceptance\nand eﬀorts until the Springer link came finally last week, I think.\nAlso, ki

In [62]:
nawAugmenter = naw.RandomWordAug(action='swap', aug_p=0.8)
augmentedReality = nawAugmenter.augment(textToScramble)
print(augmentedReality)

['Today is our dragon boat festival, in our culture Chinese, to celebrate it with all safe and great in our lives. Hope you too, to enjoy it as my wishes deepest. Thank your message to show our words to the doctor, as his next contract checking, to all of us. I got this message to see the approved message. fact In, I have received the message from the professor, to show me, this, a couple of days ago. I am very appreciated the full of support professor the, for our Springer proceedings publication During our final, discuss I told him about the new submission — the one we were waiting last since autumn, but the updates was confusing as it not included the full feedback from reviewer or maybe editor? Anyway, I believe the team, although bit delay and less communication at recent days, they really tried best for paper and cooperation. We should be grateful, I mean all of us, for the acceptance and eﬀorts until the Springer link came finally last week, I think. Also, kindly remind me pleas

In [63]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight and effective
texts = [textToScramble, augmentedReality, textGenerationOutput, augmentedText]
embeddings = model.encode(texts)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [65]:
cos_sim = cosine_similarity([embeddings[0]], embeddings[1:])
print("Cosine Similarity with Original:")
print(f"RandomWordAug: {cos_sim[0][0]:.3f}")
print(f"pipeline text-generation: {cos_sim[0][1]:.3f}")
print(f"WordNetAugmenter: {cos_sim[0][2]:.3f}")

Cosine Similarity with Original:
RandomWordAug: 0.992
pipeline text-generation: 1.000
WordNetAugmenter: 0.859
